# NN

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import json
import numpy as np

import torch


## 1. Loading the Data


In [2]:
import json

data = []

# Assuming the NDJSON data is stored in a file called 'data.ndjson'
with open("../data/domain1_train.json", 'r') as file:
    for line in file:
        data.append(json.loads(line.strip()))

# Now, the variable 'data' contains a list of dictionaries read from the file
print(data)


[{'text': [70, 746, 825, 109, 2083, 0, 2, 0, 0, 0, 9, 0, 1004, 19, 0, 0, 7, 913], 'label': 1}, {'text': [1209, 179, 1952, 4, 4959, 7, 0, 2, 978, 1522, 6, 2874, 1051, 1], 'label': 1}, {'text': [287, 3, 3330, 0, 23, 12, 13, 465, 74, 8, 0, 8, 4, 1809, 82, 8, 4180, 3378, 8, 22, 0, 0, 3149, 0, 7, 1862, 4, 2960, 607, 797, 7, 89, 0, 1927, 9, 70, 1232, 1], 'label': 1}, {'text': [0, 0, 3, 592, 19, 2, 706, 1439, 2575, 7, 2, 0, 4680, 2575, 3, 38, 2529, 1], 'label': 1}, {'text': [9, 2, 110, 12, 42, 32, 44, 361, 9, 3860, 2358, 3, 42, 32, 0, 398, 473, 6, 260, 1579, 232, 0, 4, 0, 2960, 0, 1], 'label': 1}, {'text': [9, 892, 3, 0, 0, 3, 2, 0, 811, 3, 26, 939, 13, 60, 122, 902, 16, 0, 4, 1076, 2, 1056, 963, 400, 1067, 13, 1512, 994, 4, 32, 74, 4558, 3, 338, 1883, 1198, 108, 2, 2087, 1], 'label': 1}, {'text': [0, 1115, 0, 72, 192, 64, 31, 3295, 33, 0, 1], 'label': 1}, {'text': [2, 1510, 0, 9, 30, 138, 118, 2, 0, 0, 0, 5, 0, 12, 3, 59, 4547, 4, 0, 3, 2119, 22, 40, 0, 0, 3, 0, 0, 7, 0, 0, 415, 2, 0, 1], 'l

## 2. Pre-processing

In [3]:
texts = [torch.tensor(item['text']) for item in data]
labels = [item['label'] for item in data]

# Padding sequences for NN
max_length = max([len(text) for text in texts])
texts_padded = [torch.nn.functional.pad(text, (0, max_length - len(text))) for text in texts]

## 3.Split Data

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(texts_padded, labels, test_size=0.2, random_state=42)


## 4. Neural Network (NN)

In [5]:
import torch.nn as nn
import torch.optim as optim

# Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(max_length, 50)
        self.fc2 = nn.Linear(50, 2)  # 2 classes: 0 and 1

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training
for epoch in range(10):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(zip(X_train, y_train), 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(outputs.unsqueeze(0), torch.tensor([labels]))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Print statistics
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(X_train)}")

print('Finished Training')


Epoch 1, Loss: 1.9490634953206927
Epoch 2, Loss: 0.7420288343203058
Epoch 3, Loss: 0.7056567121999829
Epoch 4, Loss: 0.6932754206439736
Epoch 5, Loss: 0.6528795883943009
Epoch 6, Loss: 0.6715846699444113
Epoch 7, Loss: 0.6770494991118902
Epoch 8, Loss: 0.6568287854877009
Epoch 9, Loss: 0.659860783175534
Epoch 10, Loss: 0.6534276894816732
Finished Training


## 4.1 Evaluation (NN)

In [6]:
from sklearn.metrics import accuracy_score

In [12]:
# Set the model to evaluation mode
net.eval()

# Lists to store predictions and true labels
true_labels = []
predicted_labels = []

# Loop over test data
with torch.no_grad():
    for inputs, labels in zip(X_test, y_test):
        outputs = net(inputs.float().unsqueeze(0))
        _, predicted = torch.max(outputs, 1)
        true_labels.append(labels)
        predicted_labels.append(predicted.item())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"NN Test Accuracy: {accuracy:.4f}")

NN Test Accuracy: 0.6144


## 5. KNN

In [7]:
from sklearn.neighbors import KNeighborsClassifier

# Convert torch tensors back to lists
X_train_knn = [text.tolist() for text in X_train]
X_test_knn = [text.tolist() for text in X_test]

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_knn, y_train)

# Test accuracy
accuracy = knn.score(X_test_knn, y_test)
print(f"KNN Accuracy: {accuracy * 100:.2f}%")


KNN Accuracy: 59.64%


## 6. Optimisation
Achieving an accuracy of \(0.61\) (or \(61\%\)) can be influenced by a variety of factors. Here are some potential reasons and corresponding solutions:

1. **Quality of Data**:
    - **Problem**: The data might be noisy, imbalanced, or it might not contain enough informative features to distinguish between classes.
    - **Solution**: Clean the data, use techniques like oversampling or undersampling for class imbalance, and feature engineering to enhance the data quality.

2. **Model Complexity**:
    - **Problem**: The architecture might be too simple to capture the patterns in the data or too complex leading to overfitting.
    - **Solution**: Try different architectures or increase the capacity of the current model. Adding more layers or neurons might help. Conversely, if overfitting is a concern, consider regularization techniques like dropout.

3. **Hyperparameters**:
    - **Problem**: Learning rate, batch size, and other hyperparameters might not be optimal.
    - **Solution**: Use techniques like grid search or random search to find a better set of hyperparameters. 

4. **Training Time**:
    - **Problem**: Not training for enough epochs or training for too many epochs (overfitting).
    - **Solution**: Use techniques like early stopping, or try training the model for more epochs. Monitoring validation loss can give insights here.

5. **Loss Function**:
    - **Problem**: The chosen loss function might not be suitable for the data distribution or problem type.
    - **Solution**: Ensure the loss function is apt for the classification task at hand. For binary classification, binary cross-entropy is often used.

6. **Data Split**:
    - **Problem**: The train-test split might have caused some data imbalance or important patterns to be missed in the training data.
    - **Solution**: Consider using techniques like stratified sampling, and cross-validation.

7. **Feature Representation**:
    - **Problem**: The way the data is represented might not be suitable for the neural network to understand.
    - **Solution**: Consider feature engineering or other representation techniques. For text data, methods like word embeddings (Word2Vec, GloVe) or TF-IDF might be useful.

8. **Baseline Comparison**:
    - **Problem**: It's essential to know the performance of a simple model on the dataset to understand if \(0.61\) is good or not.
    - **Solution**: Implement a simple model (like a logistic regression) to see its performance. If it performs similarly, then perhaps the dataset is challenging.

9. **Dataset Size**:
    - **Problem**: The dataset might be too small to train a deep neural network effectively.
    - **Solution**: Augment the dataset, or use techniques that do not require a lot of data, or try transfer learning.

Lastly, always keep the domain and context of the problem in mind. In some cases, \(61\%\) might be a significant improvement over random guessing, especially if the problem is complex or if there's a class imbalance. In other scenarios, \(61\%\) might be unsatisfactory, and more efforts in terms of data collection, preprocessing, and modeling might be required.